In [1]:
import sys
import csv
import pandas as pd
import numpy as np
import pytz
import glob
import datetime
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr

import bokeh
from bokeh.models import DatetimeTickFormatter
from bokeh.plotting import figure
from bokeh.layouts import gridplot, row, column
import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

comparing files from py-ginput-devel-for-tccon2017geoschemfiles


In [2]:
gc = xr.open_dataset('/Users/arianatribby/git/oklahoma_propane/data/geoschem_aws_downloads/GEOSChem.SpeciesConc.20170601_0000z.nc4')
vmr_colnames = pd.read_csv('/Users/arianatribby/git/oklahoma_propane/data/geoschem_aws_downloads/JL1_2017060103Z_37N_097W.vmr',sep="\t",skiprows=8).columns[0].split()
vmr_colnames_f = [x.strip() for x in vmr_colnames]
vmr = pd.read_csv('/Users/arianatribby/git/oklahoma_propane/data/geoschem_aws_downloads/JL1_2017060103Z_37N_097W.vmr',sep="\t",skiprows=9,names=['X'],header=None)


#### new vmr

In [3]:
vmr_vals = [x.split() for x in vmr.X.values]

In [4]:
ar = np.zeros(np.shape(vmr_vals))
for i in range(len(vmr_vals)):
    for j in range(len(vmr_vals[0])):
        ar[i,j] = float(vmr_vals[i][j])
    

In [5]:
vmr_f = pd.DataFrame(data=ar,columns=vmr_colnames_f)

#### old/original vmr

In [6]:
vmr_old_colnames = pd.read_csv('/Users/arianatribby/git/oklahoma_propane/data/geoschem_aws_downloads/JL1_2017060103Z_37N_097W_original.vmr',sep="\t",skiprows=7).columns[0].split()
vmr_old_colnames_f = [x.strip() for x in vmr_old_colnames]
vmr_old = pd.read_csv('/Users/arianatribby/git/oklahoma_propane/data/geoschem_aws_downloads/JL1_2017060103Z_37N_097W_original.vmr',sep="\t",skiprows=8,names=['X'],header=None)

In [7]:
vmr_old_vals = [x.split() for x in vmr_old.X.values]

In [8]:
ar_old = np.zeros(np.shape(vmr_old_vals))
for i in range(len(vmr_old_vals)):
    for j in range(len(vmr_old_vals[0])):
        ar_old[i,j] = float(vmr_old_vals[i][j])

In [9]:
vmr_old_f = pd.DataFrame(data=ar_old,columns=vmr_old_colnames_f)

#### compare new and old vmr 

In [10]:
vmr_f.columns

Index(['Altitude', 'H2O', 'CO2', 'O3', 'N2O', 'CO', 'CH4', 'O2', 'NO', 'SO2',
       'NO2', 'NH3', 'HNO3', 'OH', 'HF', 'HCl', 'HBr', 'HI', 'ClO', 'OCS',
       'H2CO', 'HOCl', 'HO2', 'H2O2', 'HONO', 'HO2NO2', 'N2O5', 'ClNO3', 'HCN',
       'CH3F', 'CH3Cl', 'CF4', 'CCl2F2', 'CCl3F', 'CH3CCl3', 'CCl4', 'COF2',
       'COFCl', 'C2H6', 'C2H4', 'C2H2', 'N2', 'CHClF2', 'COCl2', 'CH3Br',
       'CH3I', 'HCOOH', 'H2S', 'CHCl2F', 'HDO', 'SF6', 'F113', 'ClCN', 'F142b',
       'dust_m', 'PH3', 'CH3OH', 'CH3SH', 'CH3CHO', 'CH3CN', 'PAN', 'NF3',
       'ClOOCl', 'ClClO2', 'ClOClO', 'CHF3', 'f141b', 'CH3COOH', 'cirrus6',
       'cirrus15', 'C3H8', 'D2O', 'sa_venus', 'C6H6', 'C3H6', 'CH3COCH3',
       'CFH2CF3', 'n-C4H10', 'C5H8', 'LUFT'],
      dtype='object')

In [11]:
vmr_f.C2H6[0]

2.597e-09

In [12]:
vmr_f.C3H8[0]

9.16e-10

In [21]:
vmr_f.columns

Index(['Altitude', 'H2O', 'CO2', 'O3', 'N2O', 'CO', 'CH4', 'O2', 'NO', 'SO2',
       'NO2', 'NH3', 'HNO3', 'OH', 'HF', 'HCl', 'HBr', 'HI', 'ClO', 'OCS',
       'H2CO', 'HOCl', 'HO2', 'H2O2', 'HONO', 'HO2NO2', 'N2O5', 'ClNO3', 'HCN',
       'CH3F', 'CH3Cl', 'CF4', 'CCl2F2', 'CCl3F', 'CH3CCl3', 'CCl4', 'COF2',
       'COFCl', 'C2H6', 'C2H4', 'C2H2', 'N2', 'CHClF2', 'COCl2', 'CH3Br',
       'CH3I', 'HCOOH', 'H2S', 'CHCl2F', 'HDO', 'SF6', 'F113', 'ClCN', 'F142b',
       'dust_m', 'PH3', 'CH3OH', 'CH3SH', 'CH3CHO', 'CH3CN', 'PAN', 'NF3',
       'ClOOCl', 'ClClO2', 'ClOClO', 'CHF3', 'f141b', 'CH3COOH', 'cirrus6',
       'cirrus15', 'C3H8', 'D2O', 'sa_venus', 'C6H6', 'C3H6', 'CH3COCH3',
       'CFH2CF3', 'n-C4H10', 'C5H8', 'LUFT'],
      dtype='object')

In [24]:
fh = 400
fw = 400
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title='')

p.circle(vmr_f.C3H8.values*1e9,vmr_f.Altitude.values,size=5,color=colors[1],legend_label='C3 new vmr')
p.circle(vmr_f.C2H6.values*1e9,vmr_f.Altitude.values,size=5,color=colors[3],legend_label='C2 new vmr')

p.circle(vmr_old_f.C3H8.values*1e9,vmr_old_f.Altitude.values,size=5,color=colors[0],legend_label='C3 old vmr')
p.circle(vmr_old_f.C2H6.values*1e9,vmr_old_f.Altitude.values,size=5,color=colors[2],legend_label='C2 old vmr')



# p.circle(oldreggc.Met_THETA.values[0,11:23,28:43,4:8].flatten(),
#         oldreggc.SpeciesConc_C3H8.values[0,11:23,28:43,4:8].flatten()*1e9,
#         size=5, color=colors[1],legend_label='reg run c3 20160601')

# p.circle(gc2x25.Met_THETA.values[0,11:23,10:39,:].flatten(),
#          gc2x25.SpeciesConc_C3H8.values[0,11:23,10:39,:].flatten()*1e9,
#          size=5,color=colors[2],legend_label='2x25 run c3 20160701')

p.xaxis.axis_label = "hydrocarbon (ppb)"
p.yaxis.axis_label = "Altitude (km)"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.label_text_font_size = '14pt'
p.legend.location = "top_right"
# p.output_backend = "svg"
bokeh.io.show(p)

In [13]:
fh = 400
fw = 400
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title='')

p.circle(vmr_f.C3H8.values*1e9,vmr_f.index.values,size=5,color=colors[1],legend_label='C3 new vmr')
p.circle(vmr_f.C2H6.values*1e9,vmr_f.index.values,size=5,color=colors[3],legend_label='C2 new vmr')

p.circle(vmr_old_f.C3H8.values*1e9,vmr_old_f.index.values,size=5,color=colors[0],legend_label='C3 old vmr')
p.circle(vmr_old_f.C2H6.values*1e9,vmr_old_f.index.values,size=5,color=colors[2],legend_label='C2 old vmr')



# p.circle(oldreggc.Met_THETA.values[0,11:23,28:43,4:8].flatten(),
#         oldreggc.SpeciesConc_C3H8.values[0,11:23,28:43,4:8].flatten()*1e9,
#         size=5, color=colors[1],legend_label='reg run c3 20160601')

# p.circle(gc2x25.Met_THETA.values[0,11:23,10:39,:].flatten(),
#          gc2x25.SpeciesConc_C3H8.values[0,11:23,10:39,:].flatten()*1e9,
#          size=5,color=colors[2],legend_label='2x25 run c3 20160701')

p.xaxis.axis_label = "hydrocarbon (ppb)"
p.yaxis.axis_label = "index"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.label_text_font_size = '14pt'
p.legend.location = "top_right"
# p.output_backend = "svg"
bokeh.io.show(p)

### include geos chem
First grab the location nearest. Here are the coordinates of lamont -97.56, 36.69 but I am using the lat/lon in the name of the file for the vmr 

In [14]:
gc_lamont = gc.sel(lon=-97.,lat=36.6,method='nearest')

In [15]:
gc_lamontt1 = gc_lamont.isel(time=0)

In [16]:
indnum = list(range(len(gc_lamontt1.SpeciesConc_C3H8.values)))

In [17]:
fh = 400
fw = 400
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title='')
p.y_range.flipped = True
# p.circle(gc_lamontt1.SpeciesConc_C3H8.values[::-1]*1e9,
#          indnum,size=5,color=colors[1],legend_label='C3 gc')


p.circle(gc_lamontt1.SpeciesConc_C3H8.values*1e9,
         gc_lamontt1.lev.values,size=5,color=colors[1],legend_label='C3 gc')
p.circle(gc_lamontt1.SpeciesConc_C2H6.values*1e9,
         gc_lamontt1.lev.values,size=5,color=colors[3],legend_label='C2 gc')

p.xaxis.axis_label = "hydrocarbon (ppb)"
p.yaxis.axis_label = "lev"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.label_text_font_size = '14pt'
p.legend.location = "top_right"
# p.output_backend = "svg"
bokeh.io.show(p)

In [18]:
gc_lamontt1.SpeciesConc_C3H8.values[::-1]

array([1.8326249e-33, 1.0388588e-33, 6.3790676e-34, 1.2606093e-34,
       2.2332138e-35, 2.3303700e-36, 9.1981796e-37, 6.9662609e-37,
       3.0716030e-37, 5.1125169e-38, 3.7426636e-39, 5.9270161e-40,
       2.4462607e-40, 8.8842323e-43, 1.8216880e-44, 0.0000000e+00,
       3.2229865e-44, 2.3948191e-42, 1.3297342e-40, 1.1974067e-38,
       7.3718465e-37, 4.0148886e-35, 3.3296263e-33, 4.2796279e-31,
       3.1819342e-29, 1.8557606e-27, 8.3042617e-26, 2.5807433e-24,
       5.4857816e-23, 5.7404477e-22, 4.4965171e-21, 4.3633722e-20,
       6.0877200e-19, 5.7184613e-18, 3.6086823e-17, 3.6939451e-16,
       5.5053021e-15, 5.7579965e-14, 3.9827232e-13, 2.4243264e-12,
       1.1667933e-11, 4.3247590e-11, 6.9358394e-11, 8.6620433e-11,
       1.0470040e-10, 1.5627089e-10, 1.6874958e-10, 1.3891348e-10,
       1.1698338e-10, 1.0914709e-10, 9.9553400e-11, 8.9112474e-11,
       9.2796541e-11, 1.1563088e-10, 1.5318746e-10, 2.4770583e-10,
       3.6998080e-10, 4.9846144e-10, 5.8913008e-10, 6.4043565e

In [19]:
10**2

100

In [20]:
gc_lamont

<xarray.Dataset>
Dimensions:            (time: 7, lev: 72, ilev: 73, nb: 2)
Coordinates:
  * time               (time) datetime64[ns] 2017-06-01T03:00:00 ... 2017-06-...
  * lev                (lev) float64 0.9925 0.9775 0.9625 ... 2.635e-05 1.5e-05
  * ilev               (ilev) float64 1.0 0.985 0.97 ... 3.27e-05 2e-05 1e-05
    lat                float64 38.0
    lon                float64 -95.0
Dimensions without coordinates: nb
Data variables: (12/15)
    lat_bnds           (nb) float64 ...
    lon_bnds           (nb) float64 ...
    hyam               (lev) float64 ...
    hybm               (lev) float64 ...
    hyai               (ilev) float64 ...
    hybi               (ilev) float64 ...
    ...                 ...
    SpeciesConc_HCl    (time, lev) float32 ...
    SpeciesConc_H2O    (time, lev) float32 ...
    SpeciesConc_CH4    (time, lev) float32 ...
    SpeciesConc_CH3Cl  (time, lev) float32 ...
    SpeciesConc_C2H6   (time, lev) float32 ...
    SpeciesConc_C3H8   (time, lev) float32 ...
Attributes:
    title:                           GEOS-Chem diagnostic collection: Species...
    history:                         
    format:                          not found
    conventions:                     COARDS
    ProdDateTime:                    
    reference:                       www.geos-chem.org; wiki.geos-chem.org
    contact:                         GEOS-Chem Support Team (geos-chem-suppor...
    simulation_start_date_and_time:  2017-06-01 00:00:00z
    simulation_end_date_and_time:    2017-10-01 00:00:00z